In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [16]:
df = pd.read_csv('data/comp-merged.csv')
df.drop(columns=['date'], axis=1, inplace=True)
df.head()

,value_acc,value_met,abs_diff
0,0.000000,4.1,0.000000
1,26.694999,16.0,1.668437
2,39.588749,37.1,1.067082
3,4.161250,5.1,0.815931
4,24.852499,17.0,1.461912


In [17]:
X = df.drop(columns=['value_met'], axis=1)
y = df['value_met']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
svr_params = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

tuned_svr = GridSearchCV(SVR(), svr_params, cv=5, n_jobs=-1, verbose=2)
tuned_svr.fit(X_train, y_train)

print(tuned_svr.best_params_)
print(tuned_svr.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=0.001

In [21]:
model = tuned_svr.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 10.833492586374788
MSE: 117.36456161903752
MAE: 4.776586225995945


In [22]:
xgb_params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.3, 0.5]
}

tuned_xgb = GridSearchCV(xgb.XGBRegressor(), xgb_params, cv=5, n_jobs=-1, verbose=2)
tuned_xgb.fit(X_train, y_train)

print(tuned_xgb.best_params_)
print(tuned_xgb.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=5, n_e

In [23]:
model = tuned_xgb.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 6.772446747143934
MSE: 45.86603494290045
MAE: 2.417525549563105


In [24]:
rfr_params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    # 'min_samples_leaf': [1, 2, 4],
    # 'ccp_alpha': [0.0, 0.1, 0.2]
}
tuned_rfr = GridSearchCV(RandomForestRegressor(), rfr_params, cv=5, n_jobs=-1, verbose=2)
tuned_rfr.fit(X_train, y_train)

print(tuned_rfr.best_params_)
print(tuned_rfr.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END .max_depth=3, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=500; total time=   0.4s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=500; total time=   0.4s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=500; total time=   0.4s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=500; total time=   0.4s
[CV] END .max_depth=3, min_samples_split=2, n_estimators=500; total time=   0.4s
[CV] END .max_depth=3, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END .max_depth=3, min_samples_split=5, n_e

In [25]:
model = tuned_rfr.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 7.018703597175173
MSE: 49.262200184999706
MAE: 2.554187691983878
